In [2]:
from bs4 import BeautifulSoup

In [3]:
import io
import argparse

In [4]:
# Argparse declerations
parser = argparse.ArgumentParser()
# Argparse Arguments
parser.add_argument('books_list', nargs='+')
parser.add_argument('output_file', nargs='+')
parser.add_argument('toc_name')
parser.add_argument('ncx_name', nargs='+' )
parser.add_argument('--numbered_toc', dest='feature', action='store_false')
parser.add_argument('--list_toc', dest='feature', action='store_true')

#Too many arguments, I need to automate file naming. 




_StoreTrueAction(option_strings=['--list_toc'], dest='feature', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

In [5]:
#variable declerations
# counter for play order
counter = 3 
# main loop counter
mainloop = True


In [6]:
args = parser.parse_args('maze_input-Copy.txt A_Maze_Of_Death.txt MazeToc.html MazeNcx.ncx'.split())
print(args)
#args = parser.parse_args()
# I know this is sloppy, but it works to switch between numbered or non numbered TOCs
feature = args.feature
parser.set_defaults(feature=True)
print(args)

Namespace(books_list=['maze_input-Copy.txt'], feature=True, ncx_name=['MazeNcx.ncx'], output_file=['A_Maze_Of_Death.txt'], toc_name='MazeToc.html')
Namespace(books_list=['maze_input-Copy.txt'], feature=True, ncx_name=['MazeNcx.ncx'], output_file=['A_Maze_Of_Death.txt'], toc_name='MazeToc.html')


In [7]:
# Templates 
NUMTOCTEMPLATE = BeautifulSoup("<html><head><title>Table Of Contents</title></head><body><ol></ol></body</html>", "html.parser")
LISTTOCTEMPLATE = BeautifulSoup("<html><head><title>Table Of Contents</title></head><body><ul></ul></body</html>", "html.parser")

NCXTEMPLATE = BeautifulSoup('''
<?xml version='1.0' encoding='UTF-8'?>
<!DOCTYPE ncx PUBLIC '-//NISO/DTD ncx 2005-1/EN' 'http://www.daisy.org/z3986/2005/ncx-2005-1.dtd'>
<ncx xmlsn='http://www.daisy.org z3986/2005/ncx/' version='2005-1' xml:lang='en-US'>
<head>
<meta content='uid' name='dtb:uid'/>
<meta content='1' name='dtb:depth'/>
<meta content='0' name='dtb:totalPageCount'/>
<meta content='0' name='dtb:maxPageNumber'/>
</head>
<docTitle><text></text></docTitle>
<docAuthor><text></text></docAuthor>
<navMap>
</navMap>
</ncx>''', "xml")

In [9]:
'''def harvest(books_list, output_file):
    #used to gather several source files into one large file for processing
    #outbook = output_file[0]
    with open(output_file[0], "w+") as outfile:
        for bname in books_list:
            with open(bname, "r") as readfile:
                outfile.write(readfile.read() + "\n\n")"""


SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-9-7e89b3c14ba6>, line 7)

In [10]:
def tocnumbered(soup):
    '''generates a numbered,linked table of contents page'''
    global NUMTOCTEMPLATE
    finalToc = open(args.toc_name, "w+")
    for tag in reversed(soup.find_all("a", attrs={"id":True})):
        NUMTOCTEMPLATE.body.ol.insert(0,NUMTOCTEMPLATE.new_tag("a"))
        NUMTOCTEMPLATE.a["href"] = args.output_file[0] + '#' + (tag["id"])
        NUMTOCTEMPLATE.a.string = tag.h2.string
    for tag in NUMTOCTEMPLATE.find_all("a"): #this wraps the tags with <li> tags.
        tag.wrap(NUMTOCTEMPLATE.new_tag("li")) 
    print(finalToc, NUMTOCTEMPLATE.prettify())
    finalToc.close()

In [11]:
def toclist(soup):
    '''generates a linked table of contents page'''
    global LISTTOCTEMPLATE
    finalToc = open(args.toc_name, "w+")
    for tag in reversed(soup.find_all("a", attrs={"id":True})):
        LISTTOCTEMPLATE.body.ul.insert(0,LISTTOCTEMPLATE.new_tag("a"))
        LISTTOCTEMPLATE.a["href"] = args.output_file[0] + '#' + (tag["id"])
        LISTTOCTEMPLATE.a.string = tag.h2.string
    for tag in LISTTOCTEMPLATE.find_all("a"): #this wraps the tags with <li> tags.
        tag.wrap(LISTTOCTEMPLATE.new_tag("li")) 
    print(finalToc, LISTTOCTEMPLATE.prettify())
    finalToc.close()

In [12]:
def ncxmaker(soup):
    '''generates an ncx file for use in ebook creation'''
    global counter
    global NCXTEMPLATE
    for tag in reversed(soup.find_all("a", attrs={"id":True})): #finds all <a> tags with the ID attribute
        NCXTEMPLATE.navMap.insert(0, NCXTEMPLATE.new_tag("navpoint", id = tag["id"], playorder = 0))#add "navpoint tag"
        NCXTEMPLATE.navMap.navpoint.insert(2,NCXTEMPLATE.new_tag("content", src = args.output_file[0] + '#' + (tag["id"])))
        NCXTEMPLATE.navMap.navpoint.insert(0, NCXTEMPLATE.new_tag("navLabel"))
        NCXTEMPLATE.navLabel.string = tag.h2.string
        NCXTEMPLATE.navLabel.string.wrap(NCXTEMPLATE.new_tag("text"))
    for tag in NCXTEMPLATE.find_all("navpoint"): 
        tag["playorder"] = counter
        counter = counter + 1
    NCXTEMPLATE.navMap.insert(0, NCXTEMPLATE.new_tag("navpoint", id = "id", playorder = 2))
    NCXTEMPLATE.navMap.navpoint.insert(2,NCXTEMPLATE.new_tag("content", src = "" ))
    NCXTEMPLATE.navMap.navpoint.insert(0, NCXTEMPLATE.new_tag("navLabel"))
    NCXTEMPLATE.navLabel.string = ""
    NCXTEMPLATE.navLabel.string.wrap(NCXTEMPLATE.new_tag("text"))
    NCXTEMPLATE.navMap.insert(0, NCXTEMPLATE.new_tag("navpoint", id = "id", playorder = 1))
    NCXTEMPLATE.navMap.navpoint.insert(2,NCXTEMPLATE.new_tag("content", src = "" ))
    NCXTEMPLATE.navMap.navpoint.insert(0, NCXTEMPLATE.new_tag("navLabel"))
    NCXTEMPLATE.navLabel.string = ""
    NCXTEMPLATE.navLabel.string.wrap(NCXTEMPLATE.new_tag("text"))

In [13]:
def makefiles():
    '''simply takes the templates that were filled out and prints them to files'''
    finalToc = open(args.toc_name[0], "w+")
    finalNcx = open(args.ncx_name[0], "w+")
    print(finalToc, LISTTOCTEMPLATE.prettify())
    print(finalNcx, NCXTEMPLATE.prettify())
    finalToc.close()
    finalNcx.close()

In [15]:
#harvest(args.books_list,args.output_file)

In [16]:
print(args.output_file[0])

A_Maze_Of_Death.txt


In [17]:
the_book = open(args.output_file[0])

In [18]:
soup = BeautifulSoup(the_book, "lxml")


In [19]:
'''if feature is True:
    toclist(soup)
if feature is False:
    tocnumbered(soup)
    '''

'if feature is True:\n    toclist(soup)\nif feature is False:\n    tocnumbered(soup)\n    '

In [20]:
ncxmaker(soup)
toclist(soup)


(<open file 'MazeToc.html', mode 'w+' at 0x0000000004567D20>, u'<html>\n <head>\n  <title>\n   Table Of Contents\n  </title>\n </head>\n <body>\n  <ul>\n  </ul>\n </body>\n</html>')


In [21]:
makefiles()

(<open file 'M', mode 'w+' at 0x0000000004567D20>, u'<html>\n <head>\n  <title>\n   Table Of Contents\n  </title>\n </head>\n <body>\n  <ul>\n  </ul>\n </body>\n</html>')
(<open file 'MazeNcx.ncx', mode 'w+' at 0x0000000004567DB0>, u'<?xml version="1.0" encoding="utf-8"?>\n<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<!DOCTYPE ncx PUBLIC "-//NISO/DTD ncx 2005-1/EN" "http://www.daisy.org/z3986/2005/ncx-2005-1.dtd">\n<ncx version="2005-1" xml:lang="en-US" xmlsn="http://www.daisy.org z3986/2005/ncx/">\n <head>\n  <meta content="uid" name="dtb:uid"/>\n  <meta content="1" name="dtb:depth"/>\n  <meta content="0" name="dtb:totalPageCount"/>\n  <meta content="0" name="dtb:maxPageNumber"/>\n </head>\n <docTitle>\n  <text/>\n </docTitle>\n <docAuthor>\n  <text/>\n </docAuthor>\n <navMap>\n  <navpoint id="id" playorder="1">\n   <navLabel>\n    <text>\n    </text>\n   </navLabel>\n   <content src=""/>\n  </navpoint>\n  <navpoint id="id" playorder="2">\n   <navLabel>\n    <text>\n    </text>\n   </n